<a href="https://colab.research.google.com/github/kazueuglena/mimosa_img_process/blob/main/quantification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 実験番号
exp = 19

import tensorflow as tf
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import re
import shutil

# 入力フォルダと出力フォルダのパス
input_folder = "/content/drive/MyDrive/model/input_img"
output_folder = "/content/drive/MyDrive/model/data_img"
finished_path = "/content/drive/MyDrive/model/finished_path"

# 入力フォルダ内の画像ファイルを取得
image_files = os.listdir(input_folder)

# ターゲットサイズ
target_size = (350, 585)

# 画像ファイルを一括で処理
for image_file in image_files:
    # 画像ファイルのパス
    input_path = os.path.join(input_folder, image_file)

    # 画像を読み込み
    mimosa = cv2.imread(input_path)

    # 画像の拡大
    new_width = 1600
    new_height = 1200
    mimosa = cv2.resize(mimosa, (new_width, new_height))

    # 画像の切り抜きと保存 左＋600　右＋340
    imgA = mimosa[47:573, 42:322]
    imgA = cv2.resize(imgA, target_size)
    cv2.imwrite(os.path.join(output_folder, "A_" + image_file), imgA)

    imgB = mimosa[45:577, 339:629]
    imgB = cv2.resize(imgB, target_size)
    cv2.imwrite(os.path.join(output_folder, "B_" + image_file), imgB)

    imgC = mimosa[46:582, 638:932]
    imgC = cv2.resize(imgC, target_size)
    cv2.imwrite(os.path.join(output_folder, "C_" + image_file), imgC)

    imgD = mimosa[52:587, 942:1239]
    imgD = cv2.resize(imgD, target_size)
    cv2.imwrite(os.path.join(output_folder, "D_" + image_file), imgD)

    imgE = mimosa[60:592, 1241:1547]
    imgE = cv2.resize(imgE, target_size)
    cv2.imwrite(os.path.join(output_folder, "E_" + image_file), imgE)

    imgF = mimosa[566:1117, 14:303]
    imgF = cv2.resize(imgF, target_size)
    cv2.imwrite(os.path.join(output_folder, "F_" + image_file), imgF)

    imgG = mimosa[587:1129, 1240:1540]
    imgG = cv2.resize(imgG, target_size)
    cv2.imwrite(os.path.join(output_folder, "G_" + image_file), imgG)

    shutil.move(input_path, finished_path)

print("処理が完了しました。")


from posixpath import split
#画像を分類するためのプログラム


# 入力フォルダと出力フォルダのパス
input_folder = "/content/drive/MyDrive/model/data_img"

# 入力フォルダ内の画像ファイルを取得
image_files = os.listdir(input_folder)

# 画像ファイルを一括で処理
for image_file in image_files:
  match = re.search(r'_(.+?)-', image_file)
  if match:
      exp_times = match.group(1)
      input_path = os.path.join(input_folder, image_file)
      destination_folder_path = '/content/drive/MyDrive/model/data_img/'
      string = image_file
      destination_folder_path = os.path.join(destination_folder_path, str(match.group(1)))
      destination_folder_path = os.path.join(destination_folder_path, string[0] + "-" + match.group(1))
      shutil.move(input_path, destination_folder_path)

# カスタムオブジェクトを定義する
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# モデルの読み込み
model_path = "/content/drive/MyDrive/model/0722model.hdf5"
if os.path.exists(model_path):
    print("モデルファイルが存在します。")
else:
    print("モデルファイルが存在しません。")

# カスタムオブジェクトのスコープを作成してモデルを読み込む
with tf.keras.utils.custom_object_scope({'root_mean_squared_error': root_mean_squared_error}):
    model = tf.keras.models.load_model(model_path)

# ディレクトリ内の画像ファイルを取得
base_image_dir = '/content/drive/MyDrive/model/data_img/'
places = ["A", "B", "C", "D", "E", "F", "G"]

# 予測結果の格納先
predictions = {place: [] for place in places}

for place in places:
    image_dir = os.path.join(base_image_dir, str(exp), f"{place}-{exp}")
    image_files = os.listdir(image_dir)

    image_files.sort()
    # 画像ごとに予測を実行
    for file_name in image_files:
        # 画像のパスを取得
        image = os.path.join(image_dir, file_name)
        image = Image.open(image)
        image = image.convert('RGB')
        new_size = (350, 585)  # 新しいサイズ (幅, 高さ)
        image = image.resize(new_size)
        # 画像から配列に変換
        data = np.asarray(image)
        data = data.astype('float32') / 255
        # 予測の実行
        prediction = model.predict(np.expand_dims(data, axis=0))

        # 予測結果をリストに追加
        predictions[place].append(prediction[0][0])

        # 空のリストを作成
        time_list = []

        for file_name in image_files:
            # ファイル名から時間を取得
            index_of_t = file_name.find("T")  # "T"の位置を取得
            index_of_dot = file_name.find(".")
            time = file_name[index_of_t + 1: index_of_dot]
            time = f"{time[:2]}:{time[2:]}"
            # timeをリストに追加
            time_list.append(time)

            index = file_name.find("-")  # "T"の位置を取得
            index_of_T = file_name.find("T")
            day = file_name[index+ 1: index_of_T]

# time_listにはtimeの結果が格納されています
print(time_list)


# 予測結果の表示
print(f'実験番号: {exp}')
result_list = []
for place, place_predictions in predictions.items():
    print(f'場所: {place}')
    for prediction in place_predictions:
        print(f'葉の開閉度（％）: {prediction}')
        result_list.append([prediction])
        print('---')

print(predictions)
predictions = [predictions]
predictions = [value for value in predictions[0].values()]
predictions =np.array(predictions)
# 結果をDataFrameにまとめる
# timeの各要素を文字列に変換して新しいリストを作成
time_str = [str(t) for t in time]
# DataFrameのインデックスとして設定
print(time_list)
print(predictions)
ind = ["A", "B", "C", "D", "E", "F", "G"]
col = time_list
df = pd.DataFrame(predictions, index=ind, columns=col)

# gspreadライブラリをインポート　これ以下うまく行かないかも
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# 認証情報のパスを指定
credentials_path = ""  # 自分の認証情報のパスに置き換えてください
if os.path.exists(credentials_path):
    print("指定された認証情報ファイルは存在します。")
else:
    print("指定された認証情報ファイルは存在しません。")

# 認証情報を設定してGoogle Sheetsに接続
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
gc = gspread.authorize(credentials)

# スプレッドシートのキーを指定
spreadsheet_key = ""  # 自分のスプレッドシートのキーに置き換えてください
worksheet_name = "sheet1"  # 書き込むシートの名前に置き換えてください

# スプレッドシートを開く
worksheet = gc.open_by_key(spreadsheet_key).worksheet(worksheet_name)

# データをDataFrameから2次元リストに変換
data_list = df.values.tolist()

# データを書き込む範囲を指定（行数と列数）
start_row = 123  # データを書き込む開始行（2行目から書き込むと仮定）　10ずつ下げていく
start_col = 1 # データを書き込む開始列（2列目から書き込むと仮定）

start_name = start_row - 2

# データを書き込む
worksheet.update(f"A{start_name}",f'実験番号: {exp}')
worksheet.update(f"B{start_name}",f'日付: {day}')
worksheet.update(f"B{start_row}", data_list)

# 縦軸（行）のデータを書き込む
for i, index_value in enumerate(ind):
    worksheet.update(f"A{start_row+i}", [[index_value]])

start_row = start_row-1

import time

def get_column_name(index):
    # Convert index to column name
    column_name = ''
    while index >= 0:
        index, remainder = divmod(index, 26)
        column_name = chr(65 + remainder) + column_name
        index -= 1
    return column_name

# リクエスト間の待ち時間（秒）
request_interval = 2

for i, value in enumerate(col):
    column_name = get_column_name(start_col + i)
    cell_address = f"{column_name}{start_row}"
    worksheet.update(cell_address, [[value]])

    # 待ち時間を挿入
    time.sleep(request_interval)
